In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
shp = 'D:/Data/parper/data/shp/JX.shp'
shp_ee = geemap.shp_to_ee(shp)
roi = shp_ee.geometry()


Map.addLayer(roi, {}, 'roi')
Map.centerObject(roi)

In [ ]:
def maskClouds(image):
  # Select the QA band.
  QA = image.select('StateQA')
  # Make a mask to get bit 10, the internal_cloud_algorithm_flag bit.
  bitMask = 1 << 10
  # Return an image masking out cloudy areas.
  return image.updateMask(QA.bitwiseAnd(bitMask).eq(0))

def clip(image):
    return(image.clip(roi).copyProperties(image, ["system:time_start"]))

def ndvi(img):
    ndvi = img.normalizedDifference(['sur_refl_b02','sur_refl_b01']).rename('NDVI')
    Msk = ndvi.lte(1).And(ndvi.gt(0))
    ndvi = ndvi.updateMask(Msk)
    return img.addBands(ndvi).copyProperties(img, ["system:time_start"])

In [ ]:
# 为每幅影像添加时间波段
def addTimeImag(image):
    timeImage = image.metadata('system:time_start').rename('timestamp')
    timeImageMasked = timeImage.updateMask(image.mask().select(0))
    return image.addBands(timeImageMasked)

In [ ]:
def rename(image):
    return image.rename('LST').multiply(0.02).subtract(273).copyProperties(image, ["system:time_start"])

In [ ]:
collection_09a1 = ee.ImageCollection("MODIS/061/MOD09A1")\
                                    .select(['sur_refl_b02','sur_refl_b01','StateQA'])\
                                    .filterDate('2005', '2021')\
                                    .map(maskClouds)\
                                    .map(clip)

In [ ]:
collection_ndvi = collection_09a1.map(ndvi).select('NDVI').map(addTimeImag)

In [ ]:
collection_LST = ee.ImageCollection("MODIS/061/MOD11A2")\
            .filterDate('2005', '2021')\
            .map(clip)\
            .select('LST_Day_1km').map(rename).map(addTimeImag)

In [ ]:
LSTs_i = collection_LST.filter(ee.Filter.calendarRange(4, 5,'month'))
NDVIs_i = collection_ndvi.filter(ee.Filter.calendarRange(4, 5,'month'))

### 插值补充

In [ ]:
def image_join(filtered, secondary, days=30):
    # filtered 为需要插值的影像
    # secondary 为可用于插值匹配的影像
    # days为匹配时间段
    millis = ee.Number(days).multiply(1000*60*60*24)                  # 转化为毫秒
    # 区间匹配
    maxDiffFilter = ee.Filter.maxDifference(**{                       
      'difference': millis,
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })
    # 向后匹配
    lessEqFilter = ee.Filter.lessThanOrEquals(**{
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })
    # 向前匹配
    greaterEqFilter = ee.Filter.greaterThanOrEquals(**{
      'leftField': 'system:time_start',
      'rightField': 'system:time_start'
    })
    # 获取当前时间影像后days时间内包含的影像
    filter1 = ee.Filter.And(maxDiffFilter, lessEqFilter)

    join1 = ee.Join.saveAll(**{
      'matchesKey': 'after',
      'ordering': 'system:time_start',
      'ascending': False})

    join1Result = join1.apply(**{
      'primary': filtered,
      'secondary': secondary,
      'condition': filter1
    })
    # 获取当前时间影像前days时间内包含的影像
    filter2 = ee.Filter.And(maxDiffFilter, greaterEqFilter)

    join2 = ee.Join.saveAll(**{
      'matchesKey': 'before',
      'ordering': 'system:time_start',
      'ascending': True})

    join2Result = join2.apply(**{
      'primary': join1Result,
      'secondary': secondary,
      'condition': filter2
    })
    return join2Result

In [ ]:
# 插值公式  y = y1 + (y2-y1)*((t – t1) / (t2 – t1))
def interpolateImages(image):
    image = ee.Image(image)
    beforeImages = ee.List(image.get('before'))
    beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
    afterImages = ee.List(image.get('after'))
    afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()
    
    t1 = beforeMosaic.select('timestamp').rename('t1')
    t2 = afterMosaic.select('timestamp').rename('t2')
    t = image.metadata('system:time_start').rename('t')
    timeImage = ee.Image.cat([t1, t2, t])
    timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                't': timeImage.select('t'),
                't1': timeImage.select('t1'),
                't2': timeImage.select('t2'),
                })
    interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
    result = image.unmask(interpolated)
    return result.copyProperties(image, ['system:time_start'])

In [ ]:
NDVIs_i_join = image_join(collection_ndvi, collection_ndvi)

In [ ]:
NDVIs_inter = ee.ImageCollection(NDVIs_i_join.map(interpolateImages))

In [ ]:
def sg_images(images: ee.ImageCollection, window_size: int, order: int, deriv=0) -> list:
    """
    基于影像的sg滤波处理
    Args:
        images: ee.ImageCollection, 需要处理的影像集, 注意请影像集中的每张影像的波段应处理为只有一个波段
        window_size: int, 窗口大小, 最好为奇数
        order: int, 多项式阶数, 必须小于window_size
        deriv: int, 求导数的阶数, 默认为0

    Returns: list, 包含滤波处理后的image的list

    """
    half_window = window_size // 2
    order_range = ee.List.sequence(0, order)
    k_range = ee.List.sequence(-half_window, half_window)
    b = ee.Array(k_range.map(lambda k: order_range.map(lambda o: ee.Number(k).pow(o))))
    m_pi = ee.Array(b.matrixPseudoInverse())
    impulse_response = (m_pi.slice(**{'axis': 0, 'start': deriv, 'end': deriv + 1})).project([1])
    y = images.sort('system:time_start', False).toBands().toArray()
    times = images.aggregate_array('system:time_start')
    ids = images.aggregate_array('system:id')
    y1 = images.sort('system:time_start', True).toBands().toArray()
    y0 = y1.arrayGet(0)
    first_filling = y.arraySlice(0, -half_window - 1, -1).subtract(y0).abs().multiply(-1).add(y0)
    y_end = y.arrayGet(0)
    last_filling = y.arraySlice(0, 1, half_window + 1).subtract(y_end).abs().add(y_end)
    y_ext = first_filling.arrayCat(y1, 0).arrayCat(last_filling, 0)
    run_length = ee.List.sequence(0, images.size().subtract(1))
    # count = images.size()
    # y = images.toList(count)
    # times = images.aggregate_array('system:time_start')
    # ids = images.aggregate_array('system:id')
    # y0 = y.get(0)
    # first_filling = y.slice(1, half_window + 1).reverse().map(
    #     lambda e: ee.Number(e).subtract(y0).abs().multiply(-1).add(y0))
    # y_end = y.get(-1)
    # last_filling = y.slice(-half_window - 1, -1).reverse().map(
    #     lambda e: ee.Number(e).subtract(y_end).abs().add(y_end))
    # y_ext = ee.ImageCollection(first_filling.cat(y).cat(last_filling))
    # run_length = ee.List.sequence(0, y_ext.length().subtract(window_size))
    smooth = []
    for i in run_length.getInfo():
        smooth.append(ee.Image(y_ext.arraySlice(0, i, i + window_size)
                               .multiply(impulse_response).arrayReduce("sum", [0]).arrayGet([0])
                               .set({'system:time_start': times.get(i), 'system:id': ids.get(i)})))
    return smooth

In [ ]:
NDVIs_sg = sg_images(NDVIs_inter, 7, 3)